In [30]:
# Authenticate user for Google Cloud (Vertex AI) access
from google.colab import userdata
from google.colab import auth
auth.authenticate_user()
GEMINI_API_KEY=userdata.get('GEMINI_API_KEY')
GOOGLE_CLOUD_PROJECT=userdata.get('GOOGLE_CLOUD_PROJECT')


# Importing libraries :
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict

In [31]:
# creating a model
model = ChatGoogleGenerativeAI(
    model="gemini-3-pro-preview",
    project=GOOGLE_CLOUD_PROJECT,
    location="global",
    temperature=1.0,
)

In [32]:
class BlogState(TypedDict):
    # define states
    title: str
    outline: str
    content: str

In [33]:
def create_outline(state: BlogState) -> BlogState:
    """Create an outline for a blog post."""
    # fetch title
    title = state['title']

    # call LLM and generate ouline
    prompt = f'Generate a detailed outline for a blog on the topic- {title}.'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline
    return state

In [34]:
def create_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the following outline \n {outline}'

    # Generate content and save to state
    content = model.invoke(prompt).content
    state['content'] = content
    return state

In [35]:
# graph
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()
print("Graph compiled successfully with updated functions.")

In [36]:
initial_state = {'title':'raise of AI in India'}
final_state = workflow.invoke(initial_state)
print(final_state)

{'title': 'raise of AI in India', 'outline': [{'type': 'text', 'text': 'Here is a detailed outline for a blog post on the **Rise of AI in India**. This structure is designed to guide a writer from a strong hook to a compelling conclusion, covering policy, industry, and social impact.\n\n---\n\n### **Blog Title Options:**\n*   *From IT Hub to AI Powerhouse: Charting the Rise of Artificial Intelligence in India*\n*   *AI for All: How India is Uniquely Positioning Itself in the Global AI Race*\n*   *The Indian AI Revolution: Trends, Challenges, and the Road Ahead*\n\n---\n\n### **1. Introduction**\n*   **The Hook:** Start with a contrast between India’s historical reputation as the world\'s "back office" for IT support and its emerging status as a global innovation hub for Artificial Intelligence.\n*   **The Context:** Briefly mention the global AI boom (Generative AI, ChatGPT effect) and how it triggered a massive shift in the Indian tech ecosystem.\n*   **The Thesis Statement:** India i

In [39]:
final_state['content'][0]['text']

'Here is a detailed blog post based on your outline.\n\n***\n\n# From IT Hub to AI Powerhouse: Charting the Rise of Artificial Intelligence in India\n\nFor decades, India held a distinct reputation in the global technology ecosystem: the world’s "back office." If you needed code maintenance, IT support, or BPO services, you came to India. But the winds are shifting. Today, India is shedding its service-provider skin to emerge as a global innovation hub for Artificial Intelligence.\n\nThe release of ChatGPT and the subsequent global boom in Generative AI didn\'t just capture attention in Silicon Valley; it triggered a seismic shift in Bengaluru, Hyderabad, and Pune. We are no longer just maintaining the systems of the world; we are building them.\n\nIndia stands on the brink of an AI revolution. Fueled by massive digital infrastructure, the world\'s largest young workforce, and strategic government backing, the country is positioning itself not just to adopt AI, but to build solutions f